# SpaceTime Matrix demo

### Data used for this notebook
- Space-time Matrix in `Zarr` format: [download](https://figshare.com/ndownloader/files/41957250)
- Crop parcel polygons in `gpkg` format: [download](https://figshare.com/ndownloader/files/41012630)

Refer to the [tutorial](https://github.com/MotionbyLearning/sarxarray/blob/main/examples/demo_sarxarray.ipynb) of `sarxarray` to see how to generate a STM from a SLC stack.

### Setup environment
For the python environment setup, we assume you already started an independent Python environment with Python version later than 3.10.

To execute this notebook, first clone the `stm` repostory to your local and install it with `demo` extra options

```sh
git clone git@github.com:MotionbyLearning/stm.git
cd stm
pip install .[demo]
```

After installation, execute the notebook in a JupyterLab session, which can be started by running `jupyterlab` command in your command line:
```bash
jupyter-lab
```

A new tab will be opened in your default browser to execute this notebook.

In [ ]:
import numpy as np
from pathlib import Path
import stmtools

### Loading SpaceTime Matrix from CSV file

In [ ]:
# Reading example data
stm_demo = stmtools.from_csv('./data/example.csv')
stm_demo

### Loading SpaceTime Matrix from Zarr

In [ ]:
import xarray as xr
path_stm = Path('./stm.zarr')
stm_demo = xr.open_zarr(path_stm)
stm_demo = stm_demo.chunk({"space": 10000, "time":-1})
stm_demo

### STM enrichment from Polygon file

In [ ]:
# Path to the BRP polygon of NL
path_polygon = Path('../../data/brp/brpgewaspercelen_concept_2022_wgs84.gpkg')

In [ ]:
import geopandas as gpd
# Read one row and check columns
polygons_one_row = gpd.read_file(path_polygon, rows=1)
polygons_one_row

In [ ]:
import stmtools

# Data enrichment
fields_to_query = ['gewas', 'category']

# When AoI is small and less number of chunks: directly feed in the polygon file. Multi-process can be blocked by file IO
stm_demo = stm_demo.stm.enrich_from_polygon(path_polygon, fields_to_query)

# # When AoI is big and large number of chunks: First load in the polygons
# xmin, ymin, xmax, ymax = [
#         stm_demo['lon'].data.min().compute(),
#         stm_demo['lat'].data.min().compute(),
#         stm_demo['lon'].data.max().compute(),
#         stm_demo['lat'].data.max().compute(),
#     ]
# polygons = gpd.read_file(path_polygon, bbox=(xmin, ymin, xmax, ymax))
# stm_demo = stm_demo.stm.enrich_from_polygon(polygons, fields_to_query)

In [ ]:
stm_demo

### Subset by Polygons

In [ ]:
stm_demo_subset = stm_demo.stm.subset(method='polygon', polygon=path_polygon)
stm_demo_subset

### Visualize the points

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
plt.scatter(stm_demo_subset.lon.data, stm_demo_subset.lat.data, s=0.1)